In [1]:
import csv
import requests
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse, parseString


In [2]:
cc_xml = ''
with open('./Conversation_SSML.xml', 'r') as cc_file:
    cc_xml = cc_file.read()

In [3]:
conversation = []
i=1

tree = parseString(cc_xml)
for el in tree.getElementsByTagName('voice'):
    conv_part = ''
    speaker = (dict(el.attributes.items())['name'])
    for node in el.childNodes:
        if node.nodeType == node.TEXT_NODE:
            conv_part = node.data
        elif node.tagName == 'prosody':
            conv_part = conv_part + node.firstChild.data
        else: 
            for pro in node.getElementsByTagName('prosody'):
                conv_part = conv_part + pro.firstChild.data
    part = 'Agent'
    if speaker == 'en-US-JennyNeural' : 
        part = 'Agent'
    else:
        part = 'Customer'
    conversation.append({"text":conv_part, 'id':i, 'role':part, 'participantId':'{}'.format(part)})
    i = i+1
    
#print(conversation)


In [4]:
# Format Conversation for Language Service Analysis

json_d = { "displayName": "Conversation Task Example",
  "analysisInput": {
    "conversations": [
      {
        "conversationItems": conversation ,
        "modality": "text",
        "id": "conversation1",
        "language": "en"
      }
    ]
  },
  "tasks": [             
       {
        "taskName": "analyze 1",
        "kind": "ConversationalSummarizationTask",
        "parameters": {
           "summaryAspects": ["chaptertitle","issue", "resolution", "narrative"]

        }
    }
  ]
}

In [5]:
json_d

{'displayName': 'Conversation Task Example',
 'analysisInput': {'conversations': [{'conversationItems': [{'text': 'Hello, how may I help you today?',
      'id': 1,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "I really need help with my credit card, it's not working at all",
      'id': 2,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'May I please have your first and last name?',
      'id': 3,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "sure it's John, J O H N, Doh, D O E",
      'id': 4,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'Thank you Mr Doh, can you confirm the last four digits of your account number?',
      'id': 5,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "Which number? Is that the card number or the number on my statement, I don't have a statement in front of me.",
      'id': 6,
      'role': 'Customer',
      'participantId': '

## Open AI Summarization

In [15]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2023-07-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model = parser.get('openai_api','api_model')

In [16]:
response = openai.ChatCompletion.create(
  engine=model,
  temperature=0.7,
  messages = [{"role":"system","content":"You are an AI assistant that generate detailed call summary notes in list format from the user text."},{"role":"user", "content":f"{conversation}"}],
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

In [21]:
print('Response:')
print(response["choices"][0]["message"]["content"])

Response:
Call Summary:

1. Customer reported an issue with their credit card.
2. Agent asked for customer's first and last name.
3. Customer provided their name as John Doh.
4. Agent asked for the last four digits of customer's account number.
5. Customer initially asked for clarification, but eventually provided the number as 4321.
6. Agent informed customer of suspected fraud on their credit card and asked for confirmation of last purchase made.
7. Customer confirmed trying to use the card to book an AirBnB for their daughter.
8. Agent asked for confirmation of charge amount, which customer initially didn't know but eventually provided as around $2000.
9. Agent confirmed charge and marked it as approved.
10. Customer successfully made the booking and thanked the agent.
11. Customer requested to order another card for their daughter.
12. Agent asked for daughter's first and last name.
13. Customer provided daughter's name as Jane Doh.
14. Agent asked for mailing address for the new c

In [24]:
response = openai.ChatCompletion.create(
  engine=model,
  temperature=0.2,
  messages = [{"role":"system","content":"You are an AI assistant that generates extract distinct list of call participants in list format from the following user text."},{"role":"user", "content":f"{conversation}"}],
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

print('Response:')
print(response["choices"][0]["message"]["content"])

Response:
The distinct list of call participants in list format is: 

- Agent
- Customer


In [26]:
JSONprompt = '''You must extract the following information from the phone conversation below:
	1. Call reason (key: reason)
	2. Cause of incident (key: cause)
	3. Name of caller (key: caller)
	4. Account number (key: account_id)
    5. Follow-up Flag(key: followupind)
	6. A short, yet detailed summary (key: summary)

Make sure fields 1 to 4 are answered very short. Field 5 is a boolean flag which indicates whether the caller incident was resolved, 1 if follow up required, 0 if resolved.  Please answer in JSON machine-readable format, using the keys from above. Format the output as JSON object. Pretty print the JSON and make sure that is properly closed at the end.'''

In [35]:
messages = [{"role":"system","content":f"You are an AI assistant {JSONprompt} from the following user text."},{"role":"user", "content":f"{conversation}"}]


In [36]:
response = openai.ChatCompletion.create(
  engine=model,
  temperature=0.2,
  messages = messages,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

print('Response:')
print(response["choices"][0]["message"]["content"])

Response:
{
    "reason": "Credit card issue",
    "cause": "Suspected fraud on credit card",
    "caller": "John Doh",
    "account_id": "4321",
    "followupind": 0,
    "summary": "John Doh reported suspected fraud on his credit card. Agent confirmed the last purchase and marked it as approved. John Doh requested a new card for his daughter and provided her name and mailing address. Agent processed the request and informed John Doh that he can expect the card in 1 to 2 business days."
}


In [38]:
eval(response["choices"][0]["message"]["content"])

{'reason': 'Credit card issue',
 'cause': 'Suspected fraud on credit card',
 'caller': 'John Doh',
 'account_id': '4321',
 'followupind': 0,
 'summary': 'John Doh reported suspected fraud on his credit card. Agent confirmed the last purchase and marked it as approved. John Doh requested a new card for his daughter and provided her name and mailing address. Agent processed the request and informed John Doh that he can expect the card in 1 to 2 business days.'}